In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1274191,2021-05-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1274192,2021-05-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1274193,2021-05-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1274194,2021-05-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
33931,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33933,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33935,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33937,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33939,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
783119,2021-05-14,Arkansas,Arkansas,5001,2112,36.00,5000,Arkansas,AR,Arkansas,State,3017804
783121,2021-05-15,Arkansas,Arkansas,5001,2114,37.00,5000,Arkansas,AR,Arkansas,State,3017804
783123,2021-05-16,Arkansas,Arkansas,5001,2114,37.00,5000,Arkansas,AR,Arkansas,State,3017804
783125,2021-05-17,Arkansas,Arkansas,5001,2114,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1274191,2021-05-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1274192,2021-05-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1274193,2021-05-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1274194,2021-05-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-18') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
483,2021-05-18,Snohomish,Washington,37931,579.00,53061,WA,County,822083,4614.01,70.43
964,2021-05-18,Cook,Illinois,547922,10716.00,17031,IL,County,5150233,10638.78,208.07
1444,2021-05-18,Orange,California,271352,5031.00,6059,CA,County,3175692,8544.66,158.42
1923,2021-05-18,Maricopa,Arizona,545468,9963.00,4013,AZ,County,4485414,12160.93,222.12
2402,2021-05-18,Los Angeles,California,1238345,24123.00,6037,CA,County,10039107,12335.21,240.29
...,...,...,...,...,...,...,...,...,...,...,...
1273447,2021-05-18,Wheeler,Oregon,32,1.00,41069,OR,County,1332,2402.40,75.08
1273665,2021-05-18,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1273852,2021-05-18,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1274035,2021-05-18,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
483,2021-05-18,Snohomish,Washington,37931,579.00,53061,WA,County,822083,4614.01,70.43,70.43,4614.01
964,2021-05-18,Cook,Illinois,547922,10716.00,17031,IL,County,5150233,10638.78,208.07,208.07,10638.78
1444,2021-05-18,Orange,California,271352,5031.00,6059,CA,County,3175692,8544.66,158.42,158.42,8544.66
1923,2021-05-18,Maricopa,Arizona,545468,9963.00,4013,AZ,County,4485414,12160.93,222.12,222.12,12160.93
2402,2021-05-18,Los Angeles,California,1238345,24123.00,6037,CA,County,10039107,12335.21,240.29,240.29,12335.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273447,2021-05-18,Wheeler,Oregon,32,1.00,41069,OR,County,1332,2402.40,75.08,75.08,2402.40
1273665,2021-05-18,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1273852,2021-05-18,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1274035,2021-05-18,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
818900,2021-05-18,Hale,Alabama,2242,78.00,1065,AL,County,14651,15302.71,532.39,532.39,15302.71,1
891445,2021-05-18,Clarke,Alabama,3515,61.00,1025,AL,County,23622,14880.20,258.23,258.23,14880.20,2
640581,2021-05-18,Lowndes,Alabama,1407,53.00,1085,AL,County,9726,14466.38,544.93,544.93,14466.38,3
567927,2021-05-18,Etowah,Alabama,13938,353.00,1055,AL,County,102268,13628.90,345.17,345.17,13628.90,4
476496,2021-05-18,Franklin,Alabama,4262,82.00,1059,AL,County,31362,13589.69,261.46,261.46,13589.69,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207086,2021-05-18,Platte,Wyoming,633,11.00,56031,WY,County,8393,7542.00,131.06,131.06,7542.00,19
781462,2021-05-18,Converse,Wyoming,1006,17.00,56009,WY,County,13822,7278.25,122.99,122.99,7278.25,20
971787,2021-05-18,Lincoln,Wyoming,1411,12.00,56023,WY,County,19830,7115.48,60.51,60.51,7115.48,21
1081763,2021-05-18,Niobrara,Wyoming,161,2.00,56027,WY,County,2356,6833.62,84.89,84.89,6833.62,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
640581,2021-05-18,Lowndes,Alabama,1407,53.00,1085,AL,County,9726,14466.38,544.93,544.93,14466.38,3,1
818900,2021-05-18,Hale,Alabama,2242,78.00,1065,AL,County,14651,15302.71,532.39,532.39,15302.71,1,2
272272,2021-05-18,Walker,Alabama,7263,278.00,1127,AL,County,63521,11434.01,437.65,437.65,11434.01,28,3
566247,2021-05-18,Clay,Alabama,1573,57.00,1027,AL,County,13235,11885.15,430.68,430.68,11885.15,20,4
690858,2021-05-18,Greene,Alabama,926,34.00,1063,AL,County,8111,11416.59,419.18,419.18,11416.59,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689185,2021-05-18,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
947538,2021-05-18,Uinta,Wyoming,2249,13.00,56041,WY,County,20226,11119.35,64.27,64.27,11119.35,5,20
971787,2021-05-18,Lincoln,Wyoming,1411,12.00,56023,WY,County,19830,7115.48,60.51,60.51,7115.48,21,21
270994,2021-05-18,Teton,Wyoming,3771,9.00,56039,WY,County,23464,16071.43,38.36,38.36,16071.43,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.43,4614.01,20,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.43,4614.01,20,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.43,4614.01,20,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.43,4614.01,20,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,70.43,4614.01,20,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271209,2021-05-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1271210,2021-05-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1271211,2021-05-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1271212,2021-05-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
815504,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15302.71,2,1,1.00
815505,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15302.71,2,1,0.00
815506,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15302.71,2,1,0.00
815507,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15302.71,2,1,0.00
815508,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15302.71,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988071,2021-05-14,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5735.76,6,23,0.00
988072,2021-05-15,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5735.76,6,23,0.00
988073,2021-05-16,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5735.76,6,23,0.00
988074,2021-05-17,Crook,Wyoming,434,12.00,56011,WY,County,7584,5722.57,158.23,158.23,5735.76,6,23,2.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
638015,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14466.38,1,3,1.00,0.00
638016,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14466.38,1,3,0.00,0.00
638017,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14466.38,1,3,0.00,0.00
638018,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14466.38,1,3,0.00,0.00
638019,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14466.38,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635915,2021-05-14,Albany,Wyoming,4413,12.00,56001,WY,County,38880,11350.31,30.86,30.86,11448.05,23,4,7.00,0.00
635916,2021-05-15,Albany,Wyoming,4413,12.00,56001,WY,County,38880,11350.31,30.86,30.86,11448.05,23,4,0.00,0.00
635917,2021-05-16,Albany,Wyoming,4413,12.00,56001,WY,County,38880,11350.31,30.86,30.86,11448.05,23,4,0.00,0.00
635918,2021-05-17,Albany,Wyoming,4435,12.00,56001,WY,County,38880,11406.89,30.86,30.86,11448.05,23,4,22.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-18') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
325064,2021-05-18,Imperial,California,28533,729.00,6025,CA,County,181215,15745.39,402.28,402.28,15745.39,1,2,17.00,0.00
2402,2021-05-18,Los Angeles,California,1238345,24123.00,6037,CA,County,10039107,12335.21,240.29,240.29,12335.21,2,5,120.00,3.00
161790,2021-05-18,San Bernardino,California,297300,4724.00,6071,CA,County,2180085,13637.08,216.69,216.69,13637.08,3,4,39.00,0.00
569559,2021-05-18,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,0.00,0.00
72313,2021-05-18,Stanislaus,California,62336,1062.00,6099,CA,County,550660,11320.23,192.86,192.86,11320.23,5,9,26.00,1.00
33492,2021-05-18,Riverside,California,300039,4598.00,6065,CA,County,2470546,12144.64,186.11,186.11,12144.64,6,7,186.00,5.00
57525,2021-05-18,San Joaquin,California,73401,1399.00,6077,CA,County,762148,9630.81,183.56,183.56,9630.81,7,18,62.00,5.00
89670,2021-05-18,Tulare,California,49908,841.00,6107,CA,County,466195,10705.39,180.40,180.40,10705.39,8,10,47.00,1.00
32616,2021-05-18,Fresno,California,102013,1690.00,6019,CA,County,999101,10210.48,169.15,169.15,10210.48,9,14,26.00,0.00
516109,2021-05-18,Merced,California,31950,463.00,6047,CA,County,277680,11506.05,166.74,166.74,11506.05,10,8,13.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1206637,2021-05-18,Lassen,California,5716,24.00,6035,CA,County,30573,18696.24,78.50,78.50,18696.24,34,1,2.00,0.00
325064,2021-05-18,Imperial,California,28533,729.00,6025,CA,County,181215,15745.39,402.28,402.28,15745.39,1,2,17.00,0.00
690800,2021-05-18,Kings,California,23016,247.00,6031,CA,County,152940,15049.04,161.50,161.50,15049.04,11,3,-1.00,1.00
161790,2021-05-18,San Bernardino,California,297300,4724.00,6071,CA,County,2180085,13637.08,216.69,216.69,13637.08,3,4,39.00,0.00
2402,2021-05-18,Los Angeles,California,1238345,24123.00,6037,CA,County,10039107,12335.21,240.29,240.29,12335.21,2,5,120.00,3.00
203445,2021-05-18,Kern,California,109351,1376.00,6029,CA,County,900202,12147.38,152.85,152.85,12147.38,14,6,31.00,0.00
33492,2021-05-18,Riverside,California,300039,4598.00,6065,CA,County,2470546,12144.64,186.11,186.11,12144.64,6,7,186.00,5.00
516109,2021-05-18,Merced,California,31950,463.00,6047,CA,County,277680,11506.05,166.74,166.74,11506.05,10,8,13.00,0.00
72313,2021-05-18,Stanislaus,California,62336,1062.00,6099,CA,County,550660,11320.23,192.86,192.86,11320.23,5,9,26.00,1.00
89670,2021-05-18,Tulare,California,49908,841.00,6107,CA,County,466195,10705.39,180.40,180.40,10705.39,8,10,47.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
325064,2021-05-18,Imperial,California,28533,729.00,6025,CA,County,181215,15745.39,402.28,402.28,15745.39,1,2,17.00,0.00
2402,2021-05-18,Los Angeles,California,1238345,24123.00,6037,CA,County,10039107,12335.21,240.29,240.29,12335.21,2,5,120.00,3.00
161790,2021-05-18,San Bernardino,California,297300,4724.00,6071,CA,County,2180085,13637.08,216.69,216.69,13637.08,3,4,39.00,0.00
569559,2021-05-18,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,0.00,0.00
72313,2021-05-18,Stanislaus,California,62336,1062.00,6099,CA,County,550660,11320.23,192.86,192.86,11320.23,5,9,26.00,1.00
33492,2021-05-18,Riverside,California,300039,4598.00,6065,CA,County,2470546,12144.64,186.11,186.11,12144.64,6,7,186.00,5.00
57525,2021-05-18,San Joaquin,California,73401,1399.00,6077,CA,County,762148,9630.81,183.56,183.56,9630.81,7,18,62.00,5.00
89670,2021-05-18,Tulare,California,49908,841.00,6107,CA,County,466195,10705.39,180.40,180.40,10705.39,8,10,47.00,1.00
32616,2021-05-18,Fresno,California,102013,1690.00,6019,CA,County,999101,10210.48,169.15,169.15,10210.48,9,14,26.00,0.00
516109,2021-05-18,Merced,California,31950,463.00,6047,CA,County,277680,11506.05,166.74,166.74,11506.05,10,8,13.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,402.28,15745.39,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,402.28,15745.39,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,402.28,15745.39,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,402.28,15745.39,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,402.28,15745.39,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4262,05/14/21,Lassen,5706,24.00,18663.53,78.50,78.50,18696.24,34,1,5.00,0.00
4263,05/15/21,Lassen,5706,24.00,18663.53,78.50,78.50,18696.24,34,1,0.00,0.00
4264,05/16/21,Lassen,5706,24.00,18663.53,78.50,78.50,18696.24,34,1,0.00,0.00
4265,05/17/21,Lassen,5714,24.00,18689.69,78.50,78.50,18696.24,34,1,8.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,402.28,15745.39,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,402.28,15745.39,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,402.28,15745.39,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,402.28,15745.39,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,402.28,15745.39,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4348,05/14/21,Merced,31919,461.00,11494.89,166.02,166.74,11506.05,10,8,5.00,0.00
4349,05/15/21,Merced,31919,461.00,11494.89,166.02,166.74,11506.05,10,8,0.00,0.00
4350,05/16/21,Merced,31919,461.00,11494.89,166.02,166.74,11506.05,10,8,0.00,0.00
4351,05/17/21,Merced,31937,463.00,11501.37,166.74,166.74,11506.05,10,8,18.00,2.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)